# TP : apprentissage multimodal


Dans ce TP, nous allons utiliser le modèle d'apprentissage, FashionCLIP, pré-entraîné sur des images ainsi que des descriptions en langage naturel. Plus particulièrement, nous allons considérer deux cas d'usage :

*   **Moteur de recherche d'images :** il s'agit de trouver, à partir d'une requête en langage naturel, l'image correspondante.

*   **Classification zero-shot :** il s'agit simplement de construire un classifieur d'images (faire correspondre un label à une image).



## Dataset

Nous allons dans un premier temps télécharger les données. Celles-ci provienennt de [Kaggle](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

In [1]:
%%capture
!pip install gdown
!gdown "1igAuIEW_4h_51BG1o05WS0Q0-Cp17_-t&confirm=t"
!unzip data

### Modèle FashionCLIP

Nous allons également télécharger le modèle pré-entraîné.

In [2]:
%%capture
!pip install -U fashion-clip

In [3]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [4]:
%%capture
fclip = FashionCLIP('fashion-clip')

FashionCLIP, à l'instar de CLIP, crée un espace vectoriel partagé pour les images et le texte. Cela permet de nombreuses applications, telles que la recherche (trouver l'image la plus similaire à une requête donnée) ou la classification zero-shot.

Il y a principalement deux composants : un encodeur d'image (pour générer un vecteur à partir d'une image) et un encodeur de texte (pour générer un vecteur à partir d'un texte).










<img src="https://miro.medium.com/v2/resize:fit:1400/0*FLNMtW6jK51fm7Og"  width="400">



Nous allons télécharger les données que nous allons ensuite nettoyer.

In [5]:
articles = pd.read_csv("data_for_fashion_clip/articles.csv")

# Supprimer les éléments ayant la même description
subset = articles.drop_duplicates("detail_desc").copy()

# Supprimer les images dont la catégrie n'est pas renseignée
subset = subset[~subset["product_group_name"].isin(["Unknown"])]

# Garder seulement les descriptions dont la longueur est inférieure à 40 tokens
subset = subset[subset["detail_desc"].apply(lambda x : 4 < len(str(x).split()) < 40)]

# Supprimer les articles qui ne sont pas suffisamment fréquents dans le jeu de données
most_frequent_product_types = [k for k, v in dict(Counter(subset["product_type_name"].tolist())).items() if v > 10]
subset = subset[subset["product_type_name"].isin(most_frequent_product_types)]

subset.head(3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,176754003,176754,2 Row Braided Headband (1),74,Hair/alice band,Accessories,1010016,Solid,17,Yellowish Brown,...,Hair Accessories,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Two-strand hairband with braids in imitation s...
3,189634031,189634,Long Leg Leggings,273,Leggings/Tights,Garment Lower body,1010016,Solid,93,Dark Green,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Leggings in stretch jersey with an elasticated...


In [30]:
subset.head(30)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,176754003,176754,2 Row Braided Headband (1),74,Hair/alice band,Accessories,1010016,Solid,17,Yellowish Brown,...,Hair Accessories,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Two-strand hairband with braids in imitation s...
3,189634031,189634,Long Leg Leggings,273,Leggings/Tights,Garment Lower body,1010016,Solid,93,Dark Green,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Leggings in stretch jersey with an elasticated...
4,194270044,194270,HELENA 2-pack tanktop,253,Vest top,Garment Upper body,1010016,Solid,51,Light Pink,...,Young Girl Jersey Basic,I,Children Sizes 134-170,4,Baby/Children,79,Girls Underwear & Basics,1002,Jersey Basic,Tops in soft organic cotton jersey.
5,203027047,203027,Linni tee (1),255,T-shirt,Garment Upper body,1010017,Stripe,10,White,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Short-sleeved top in jersey with sewn-in turn-...
6,212042070,212042,Mimmi sneaker,94,Sneakers,Shoes,1010016,Solid,10,White,...,Divided Shoes,D,Divided,2,Divided,52,Divided Accessories,1020,Shoes,Cotton trainers with closed lacing and a loop ...
8,215303001,215303,Coolio sunglasses,81,Sunglasses,Accessories,1010016,Solid,17,Yellowish Brown,...,Sunglasses,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Sunglasses with plastic frames and UV-protecti...
9,216081011,216081,Norling Knit,245,Cardigan,Garment Upper body,1010010,Melange,8,Dark Grey,...,Tops Knitwear DS,D,Divided,2,Divided,58,Divided Selected,1003,Knitwear,Cardigan in a bouclé knit made from a wool ble...
10,218829015,218829,Paris glove.,71,Gloves,Accessories,1010016,Solid,17,Yellowish Brown,...,Gloves/Hats,C,Ladies Accessories,1,Ladieswear,65,Womens Big accessories,1019,Accessories,"Gloves in soft, supple leather. Lined."
11,228257001,228257,20 den 2p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with an elasticated waist. 20 denier.


In [22]:
subset.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [6]:
subset.to_csv("subset_data.csv", index=False)
f"Il y a {len(subset)} éléments dans le dataset"

'Il y a 3104 éléments dans le dataset'

## Moteur de recherche d'images

Constuire un moteur de recherche qui permet, à partir d'une description en langage naturel, de récupérer l'image correspondante. Mesurer ses performances (précision).

<img src="https://miro.medium.com/v2/resize:fit:1400/1*cnKHgLAumVyuHuK9pkqr7A.gif"  width="800">


In [8]:
images = ["data_for_fashion_clip/" + str(k) + ".jpg" for k in subset["article_id"].tolist()]
texts = subset["detail_desc"].tolist()

# Créer les représentations vectorielles (embeddings) des images et des descriptions.
image_embeddings = fclip.encode_images(images, batch_size=32)
text_embeddings = fclip.encode_text(texts, batch_size=32)

100%|██████████| 97/97 [02:49<00:00,  1.75s/it]


Map:   0%|          | 0/3104 [00:00<?, ? examples/s]

100%|██████████| 97/97 [00:56<00:00,  1.72it/s]


In [9]:
print(image_embeddings.shape)
print(text_embeddings.shape)

(3104, 512)
(3104, 512)


In [42]:
def find_closest_image(text_embeddings, image_embeddings,images, text_idx=0, top_k=5):
    embedded_txt = text_embeddings[text_idx]
    # pour chaque image, calculer cosine similarity avec embedded_txt
    similarities = []
    for image_embedding in image_embeddings:
        similarities.append(np.dot(embedded_txt, image_embedding) / (np.linalg.norm(embedded_txt) * np.linalg.norm(image_embedding)))
    # recuperer les top_k images les plus proches
    closest_image_idxs = np.argsort(similarities)[::-1][:top_k]
    return closest_image_idxs

In [43]:
# function to compute the accuracy of the model
def compute_accuracy(text_embeddings, image_embeddings, images):
    accuracy = 0
    for idx, text_embedding in enumerate(text_embeddings):
        closest_image_idxs = find_closest_image(text_embeddings, image_embeddings, images, text_idx=idx, top_k=5)
        closest_images = [images[k] for k in closest_image_idxs]
        if images[idx] in closest_images:
            accuracy += 1
    return accuracy / len(text_embeddings)

In [44]:
# compute the accuracy
accuracy = compute_accuracy(text_embeddings, image_embeddings, images)
print(f"Accuracy: {accuracy}")

Accuracy: 0.541881443298969


# Classification zero-shot

Construite un classsifieur d'images (prédire le label d'une image). Mesurer ses performances.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*No6ZONpQMIcfFaNMOI5oNw.gif"  width="800">



In [46]:
def find_closest_label(image_embeddings, text_embeddings, labels, image_idx=0, top_k=5):
    embedded_img = image_embeddings[image_idx]
    # pour chaque texte, calculer cosine similarity avec embedded_img
    similarities = []
    for text_embedding in text_embeddings:
        similarities.append(np.dot(embedded_img, text_embedding) / (np.linalg.norm(embedded_img) * np.linalg.norm(text_embedding)))
    # recuperer les top_k labels les plus proches
    closest_label_idxs = np.argsort(similarities)[::-1][:top_k]
    return [labels[k] for k in closest_label_idxs]

def compute_label_accuracy(image_embeddings, text_embeddings, labels):
    accuracy = 0
    for idx, image_embedding in enumerate(image_embeddings):
        closest_labels = find_closest_label(image_embeddings, text_embeddings, labels, image_idx=idx, top_k=5)
        if labels[idx] in closest_labels:
            accuracy += 1
    return accuracy / len(image_embeddings)

# compute the label accuracy
label_accuracy = compute_label_accuracy(image_embeddings, text_embeddings, texts)
print(f"Label Accuracy: {label_accuracy}")

Label Accuracy: 0.5676546391752577
